In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:65% !important; }</style>"))

# Packaging line simulation

## Load Python libraries

In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
# SNS styles
sns.set(style="darkgrid")
sns.set_palette("cubehelix")
import random
import simpy
import statistics
from scipy import stats
from dataclasses import dataclass
from typing import List, Dict, Callable, Union, Any, Tuple
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## The simulation algorithm in words

The process that I am going to simulate is that of a brewery packaging line. This line consists out of 9 machines ($M_m$) and 8 buffers ($B$) between the machines. There is an unlimited amount of units coming into the first machine and there is an infinite buffer size after the last machine.

A complete run through of the line is described below:A
1. Unlimited amount of units get fed into the first machine at its processing speed.
2. __Depalletizer (Standard Machine)__:
 - Unloads units at average speed $s_m$ per different state (running and different maintenance states)
 - Has a random distribution of states and the mean time to repair
 - Tailback and lack due to full or empty buffer respectively (first machine never has lack, last machine never has tailback)
3. __Buffer 1 (Standard Buffer)__:
 - Receives units from $M_m$
 - Loses units from $M_{m-1}$
 - Is full or empty, causing tailback and/or lack
  - capacity$B_t$ = capacity$B_{t-1}$ + Output $M_m$ - Input $M_{m+1}$, with capacity$B_t$ $\leq$ Max$B$
4. __Filler__:
 - Standard Machine
5. __Buffer 2__:
 - Standard Buffer
6. __Pasteurizer__:
 - Units need to pasteurize for x amount of minutes before exiting the pasteurizer
  - It can take all units delivered while pasteurizing
  - After x minutes units leave at the same rate they arrive
 - Standard Machine
7. __Repeat Standard Buffer, Standard Machine__

## Parameters for each machine and total time

In [3]:
# Configuration
class MachineConfigError(Exception):
    pass

@dataclass
class State:
    """
    Every machine has a few different states that it can operate in. Basically it is dictates 
    the speeds that the machine produces the products with and for how long.
    
    This class calls variables as given in the respective machine parameters.
    """
    id: int
    transitions: Dict[int,float]
    speed:Callable
    duration:Callable
    
    def __post_init__(self) -> None:
        self.state_population = list(self.transitions.keys())
        self.weights = self.transitions.values()
    
    def next_state_id(self) -> int:  
        """
        Calls the next state the machine will be in 
        """
        return random.choices(self.state_population, self.weights, k=1)[0]
    
    def new_speed(self) -> Union[float, int]:  
        """
        Calls the new speed that the machine will produce units with
        """
        return self.speed.new()

    def new_duration(self) -> int:  
        """
        Calls the duration of the following state.
        """
        return int(self.duration.new())
    
@dataclass
class MachineConfig:
    """
    This is what configures the machine to run at a certain speed given 
    the current state for a chosen duration.
    """
    name:str
    states:List[State]
    
    def state(self, state_id:int) -> State:
        """
        Searches for the next state called from the possibilities.
        """
        try:
            return next((state for state in self.states if state.id == state_id), )
        except StopIteration:
            raise MachineConfigError(f"Config error in machines, since state {state_id} is not found in machine {self.name}")

    def next_state_id(self, current_state:int) -> int:
        """
        Returns the next state called from the possibilities.
        """
        return self.state(current_state).next_state_id()

    def new_speed(self, current_state:int) -> float:
        """
        Returns the new speed for producing the units called from the distribution of possible speeds.
        """
        return self.state(current_state).new_speed()
    
    def new_duration(self, current_state:int) -> int:
        """
        Returns the new duration that the machine will be in the state called from the distribution of possible speeds.
        """
        return max([1, self.state(current_state).new_duration()])
    
class random_generator:
    """
    This ensures that a random value can be picked when asked for.
    """
    def __init__(self, function, **kwargs) -> None:
        self.function = function
        self.kwargs = kwargs
        
    def new(self) -> Union[float, int]:
        """
        Using the given functiona and keyword arguments, a random value is returned.
        """
        return self.function(**self.kwargs)[0]

In [4]:
# MACHINE PARAMETERS 
# id: the identification of the state, 
# transitions: the probability of arriving in each state, 
# speed: the distribution function to get the speed of the machine,
# duration: the distribution function to get the duration in current state
###################################################################################
# Depalletizer
# State 0
depal_state_0 = State(0, 
                       {128:0.886, 1024:0.108, 2048:0.006},
                       random_generator(lambda: [0]), 
                       random_generator(stats.expon.rvs , **{'loc':1, "scale":6.829, "size":1}))
# State 128
depal_speed_prob128 = {0.0: 0.02, 300.0: 0.01, 450.0: 0.04, 600.0: 0.1, 750.0: 0.33, 900.0: 0.27, 1050.0: 0.2, 1200.0: 0.04}
depal_state_128 = State(128, 
                         {1024:0.377, 2048:0.131, 0: 0.492},
                        random_generator(random.choices, **{"population":list(depal_speed_prob128.keys()), "weights":depal_speed_prob128.values(), "k":1 }),
                        random_generator(stats.lognorm.rvs, **{'s':1.595, "loc":-0.0427, "scale":21.008, "size":1}))
# State 1024
depal_speed_prob1024 = {0.0: 0.81, 150.0: 0.02, 300.0: 0.02, 450.0: 0.02, 600.0: 0.02, 750.0: 0.04, 900.0: 0.04, 1050.0: 0.03}
depal_state_1024 = State(1024, 
                          {128:0.693, 0:0.306, 2048:0.001},
                         random_generator(random.choices, **{"population":list(depal_speed_prob1024.keys()), "weights":depal_speed_prob1024.values(), "k":1 }),
                         random_generator(stats.expon.rvs, **{"loc":1, "scale":7.418, "size":1}))
# State 2048
depal_speed_prob2048 = {0.0: 0.33, 150.0: 0.09, 300.0: 0.09, 450.0: 0.08, 600.0: 0.05, 750.0: 0.16, 900.0: 0.09, 1050.0: 0.1, 1200.0: 0.01}
depal_dur_prob2048 = {1: 128.0, 6: 6.0, 3: 7.0, 9: 3.0, 15: 1.0, 10: 1.0, 4: 5.0, 2: 8.0, 8: 2.0, 5: 5.0, 7: 3.0, 11: 3.0, 21: 1.0, 13: 1.0, 83: 1.0}
depal_state_2048 = State(2048, 
                          {128:0.897, 0:0.092, 1024:0.011},
                         random_generator(random.choices, **{"population":list(depal_speed_prob2048.keys()), "weights":depal_speed_prob2048.values(), "k":1 }),
                         random_generator(random.choices, **{"population":list(depal_dur_prob2048.keys()), "weights":depal_dur_prob2048.values(), "k":1 }))
# Configuration
Depalletizer = MachineConfig("Depalletizer", states = [depal_state_0, depal_state_128, depal_state_1024, depal_state_2048])

###################################################################################
# Filler
# State 0
filler_state_0 = State(0, 
                       {128:0.371, 1024:0.263, 32768:0.193, 16384:0.172},
                       random_generator(lambda: [0]), 
                       random_generator(stats.expon.rvs , **{'loc':1., "scale":8.075, "size":1}))
# State 128
filler_speed_prob128 = {841.0: 0.11, 916.0: 0.01, 999.0: 0.01, 1200.0: 0.87}
filler_state_128 = State(128, 
                         {1024:0.798, 16384:0.104, 0:0.080, 32768:0.018},
                        random_generator(random.choices, **{"population":list(filler_speed_prob128.keys()), "weights":filler_speed_prob128.values(), "k":1 }),
                        random_generator(stats.lognorm.rvs, **{'s':1.208, "loc":-0.078, "scale":10.795, "size":1}))
# State 1024
filler_speed_prob1024 = {0.0: 0.74, 208.0: 0.2, 721.0: 0.02, 841.0: 0.02, 1200.0: 0.03}
filler_state_1024 = State(1024, 
                          {128:0.853, 0:0.063, 16384:0.048, 32768:0.063},
                         random_generator(random.choices, **{"population":list(filler_speed_prob1024.keys()), "weights":filler_speed_prob1024.values(), "k":1 }),
                         random_generator(stats.expon.rvs, **{"loc":1, "scale":3.106, "size":1}))
# State 16384
filler_speed_prob16384 = {208.0: 0.98, 1200.0: 0.02}
filler_state_16384 = State(16384, 
                           {128:0.549, 0:0.27, 1024:0.172, 32768:0.009},
                          random_generator(random.choices, **{"population":list(filler_speed_prob16384.keys()), "weights":filler_speed_prob16384.values(), "k":1 }),
                          random_generator(stats.expon.rvs, **{"loc":1, "scale":5.558, "size":1}))
# State 32768                       
filler_speed_prob32768 = {208.0: 0.89, 333.0: 0.11}
filler_state_32768 = State(32768, 
                           {0:0.364, 128:0.364, 1024:0.15, 16384:0.122},
                          random_generator(random.choices, **{"population":list(filler_speed_prob32768.keys()), "weights":filler_speed_prob32768.values(), "k":1 }),
                          random_generator(stats.pareto.rvs, **{'b':1, "loc":0, "scale":1, "size":1}))
# Configuration
Filler = MachineConfig("Filler", states = [filler_state_0, filler_state_128, filler_state_1024, filler_state_16384, filler_state_32768])

###################################################################################
#Pasteurizer
# State 0
past_dur_prob0 = {1:0.143, 4:0.071, 37:0.071, 41:0.071, 47:0.071, 73:0.071, 117:0.071, 119:0.071, 140:0.071, 156:0.071, 170:0.071, 259:0.071, 294:0.071,}
past_state_0 = State(0, 
                     {1024:0.56, 128:0.44},
                     random_generator(lambda: [0]), 
                     random_generator(random.choices, **{"population":list(past_dur_prob0.keys()), "weights":past_dur_prob0.values(), "k":1 }))
# State 128
past_dur_prob128 = {0: 1.0, 137: 1.0, 3: 11.0, 1: 18.0, 5: 4.0, 2: 8.0, 8: 1.0, 31: 1.0, 155: 1.0, 233: 2.0, 72: 2.0, 302: 1.0, 66: 1.0, 416: 1.0, 148: 1.0, 200: 1.0, 237: 1.0, 238: 1.0, 354: 1.0, 383: 2.0, 422: 1.0, 192: 1.0, 48: 1.0, 78: 1.0, 136: 1.0, 15: 1.0, 111: 1.0, 21: 1.0, 427: 1.0, 92: 1.0, 10: 2.0, 258: 1.0, 335: 1.0, 195: 1.0, 260: 1.0, 84: 1.0, 12: 1.0, 36: 1.0, 292: 1.0, 184: 1.0, 198: 1.0, 301: 1.0, 432: 2.0, 26: 1.0, 9: 1.0, 149: 1.0, 169: 1.0, 396: 1.0, 408: 1.0, 230: 1.0, 53: 1.0, 472: 1.0, 467: 1.0, 196: 1.0, 177: 1.0, 295: 1.0, 244: 1.0, 6: 2.0, 62: 1.0, 220: 1.0, 251: 1.0, 150: 2.0, 50: 1.0, 346: 1.0, 466: 1.0, 449: 1.0, 229: 1.0, 37: 1.0, 114: 1.0, 178: 1.0, 186: 1.0, 147: 1.0, 209: 1.0, 59: 2.0, 429: 1.0, 35: 1.0, 4: 2.0, 7: 1.0, 282: 1.0, 372: 1.0, 190: 1.0, 306: 1.0, 158: 1.0, 141: 1.0, 363: 1.0, 98: 1.0, 250: 1.0, 373: 1.0, 475: 1.0, 166: 1.0, 240: 1.0, 14: 1.0, 326: 1.0, 144: 1.0, 417: 1.0, 23: 1.0, 67: 1.0, 73: 1.0, 234: 1.0, 356: 1.0, 457: 1.0, 345: 1.0}
past_speed_prob128 = {630: 0.3, 833: 0.088, 1291: 0.612}
past_state_128 = State(128, 
                       {1024:0.923, 0:0.077},
                       random_generator(random.choices, **{"population":list(past_speed_prob128.keys()), "weights":past_speed_prob128.values(), "k":1 }),
                       random_generator(random.choices, **{"population":list(past_dur_prob128.keys()), "weights":past_dur_prob128.values(), "k":1 }))
# State 1024
past_speed_prob1024 = {630: 0.286, 833: 0.015, 1291: 0.699}
past_state_1024 = State(1024, 
                        {128:0.954, 0:0.046},
                        random_generator(random.choices, **{"population":list(past_speed_prob1024.keys()), "weights":past_speed_prob1024.values(), "k":1 }),
                        random_generator(stats.pareto.rvs, **{'b':1, "loc":0, "scale":1, "size":1}))
# Configuration
Pasteurizer = MachineConfig("Pasteurizer", states = [past_state_0, past_state_128, past_state_1024])

###################################################################################
# Labeler 1
# State 0
labeler1_state_0 = State(0, 
                        {128:0.794, 4: 0.206},
                        random_generator(lambda: [0]), 
                        random_generator(stats.lognorm.rvs , **{'s': 1.209, 'loc':-0.012, "scale":2.678, "size":1}))
# State 4
labeler1_speed_prob4 = {0.0: 0.28, 180.0: 0.09, 181.0: 0.3, 182.0: 0.28, 183.0: 0.05}
labeler1_state_4 = State(4, 
                        {128:0.798, 0: 0.202},
                        random_generator(random.choices, **{"population":list(labeler1_speed_prob4.keys()), "weights":labeler1_speed_prob4.values(), "k":1 }), 
                        random_generator(stats.expon.rvs , **{'loc':1, "scale":1.887, "size":1}))
# State 128
labeler1_speed_prob128 = {332.0: 0.01, 333.0: 0.07, 334.0: 0.09, 335.0: 0.05, 336.0: 0.02, 584.0: 0.02, 585.0: 0.07, 586.0: 0.12, 587.0: 0.1, 588.0: 0.07, 589.0: 0.04, 590.0: 0.02, 663.0: 0.01, 664.0: 0.04, 665.0: 0.06, 666.0: 0.07, 667.0: 0.05, 668.0: 0.03, 669.0: 0.03, 670.0: 0.02}
labeler1_state_128 = State(128, 
                          {4:0.503, 0: 0.497},
                          random_generator(random.choices, **{"population":list(labeler1_speed_prob128.keys()), "weights":labeler1_speed_prob128.values(), "k":1 }),
                          random_generator(stats.beta.rvs, **{'a':0.645, 'b':57.206, "loc":1, "scale":4601.68, "size":1}))
# Configuration
Labeler1 = MachineConfig("Labeler1", states = [labeler1_state_0, labeler1_state_4, labeler1_state_128])

###################################################################################
# Labeler 2
# State 0
labeler2_state_0 = State(0, 
                        {128:0.794, 4: 0.206},
                        random_generator(lambda: [0]), 
                        random_generator(stats.expon.rvs , **{"loc":1, "scale":7.03, "size":1}))
# State 4
labeler2_speed_prob4 = {0.0: 0.27, 215.0: 0.13, 216.0: 0.41, 217.0: 0.16, 218.0: 0.02}
labeler2_state_4 = State(4, 
                        {128:0.673, 0: 0.327},
                        random_generator(random.choices, **{"population":list(labeler2_speed_prob4.keys()), "weights":labeler2_speed_prob4.values(), "k":1 }), 
                        random_generator(stats.expon.rvs , **{'loc':1, "scale":1.055, "size":1}))
# State 128
labeler2_speed_prob128 = {343.0: 0.04, 344.0: 0.12, 345.0: 0.08, 346.0: 0.03, 578.0: 0.02, 579.0: 0.08, 580.0: 0.14, 581.0: 0.1, 582.0: 0.06, 583.0: 0.04, 584.0: 0.04, 585.0: 0.01, 663.0: 0.03, 664.0: 0.08, 665.0: 0.05, 666.0: 0.03, 667.0: 0.03, 668.0: 0.02} 
labeler2_state_128 = State(128, 
                          {4:0.603, 0:0.397},
                          random_generator(random.choices, **{"population":list(labeler2_speed_prob128.keys()), "weights":labeler2_speed_prob128.values(), "k":1 }),
                          random_generator(stats.beta.rvs, **{'a':0.790, 'b':150.314, "loc":-3.840e-25, "scale":47684.3, "size":1}))
# Configuration
Labeler2 = MachineConfig("Labeler2", states = [labeler2_state_0, labeler2_state_4, labeler2_state_128])

###################################################################################
# Capper
# State 0
gpi_state_0 = State(0, 
                    {128:1}, 
                    random_generator(lambda: [0]),
                    random_generator(stats.expon.rvs, **{'loc':1., "scale":4.078, "size":1}))
# State 128
gpi_speed_prob128 = {0.0: 0.01, 420.0: 0.13, 600.0: 0.18, 840.0: 0.46, 1080.0: 0.02, 1200.0: 0.11, 1260.0: 0.01, 1320.0: 0.02, 1440.0: 0.07}
gpi_state_128 = State(128, 
                      {0:1}, 
                      random_generator(random.choices, **{"population":list(gpi_speed_prob128.keys()), "weights":gpi_speed_prob128.values(), "k":1 }),
                      random_generator(stats.lognorm.rvs, **{'s':1.406, "loc":-0.0136, "scale":8.660, "size":1}))
# Configuration
Capper = MachineConfig("Capper", states = [gpi_state_0, gpi_state_128])

###################################################################################
# Packer
# State 0
sn64_state_0 = State(0, 
                     {128:1}, 
                     random_generator(lambda: [0]),
                     random_generator(stats.expon.rvs, **{'loc':1., "scale":5.864, "size":1}))
# State 128
sn64_speed_prob128 = {0.0: 0.03, 390.0: 0.08, 480.0: 0.03, 510.0: 0.04, 540.0: 0.18, 550.0: 0.01, 570.0: 0.01, 600.0: 0.04, 630.0: 0.02, 660.0: 0.1, 690.0: 0.03, 720.0: 0.23, 750.0: 0.02, 780.0: 0.07, 900.0: 0.07, 1000.0: 0.01, 1050.0: 0.02}
sn64_state_128 = State(128, 
                       {0:1}, 
                       random_generator(random.choices, **{"population":list(sn64_speed_prob128.keys()), "weights":sn64_speed_prob128.values(), "k":1 }),
                       random_generator(stats.lognorm.rvs, **{'s':1.59, "loc":-0.0136, "scale":10.040, "size":1}))
# Configuration
Packer = MachineConfig("Packer", states = [sn64_state_0, sn64_state_128])

###################################################################################
# Palletizer 1
# State 0
pal1_state_0 = State(0, 
                     {128:1}, 
                     random_generator(lambda: [0]),
                     random_generator(stats.expon.rvs, **{'loc':1, "scale":9.135, "size":1}))
# State 128
pal1_speed_prob128 = {0.0: 0.57, 1200: 0.43}
pal1_state_128 = State(128, 
                       {0:1}, 
                       random_generator(random.choices, **{"population":list(pal1_speed_prob128.keys()), "weights":pal1_speed_prob128.values(), "k":1 }),
                       random_generator(stats.beta.rvs, **{'a':0.614, 'b': 365.763, 'loc':-4.511e-28, "scale":16265.5, "size":1}))
# Configuration
Palletizer1 = MachineConfig("Palletizer_1", states = [pal1_state_0, pal1_state_128])

###################################################################################
# Palletizer 2
# State 0
pal2_state_0 = State(0, 
                     {128:1}, 
                     random_generator(lambda: [0]),
                     random_generator(stats.t.rvs, **{'df':0.806, 'loc':3.339, "scale":2.5, "size":1}))
# State 128
pal2_speed_prob128 = {0.0: 0.63, 1200: 0.37}
pal2_state_128 = State(128, 
                       {0:1}, 
                       random_generator(random.choices, **{"population":list(pal2_speed_prob128.keys()), "weights":pal2_speed_prob128.values(), "k":1 }),
                       random_generator(stats.t.rvs, **{'df':0.579, 'loc':8.931, "scale":13.984, "size":1}))
# Configuration
Palletizer2 = MachineConfig("Palletizer_2", states = [pal2_state_0, pal2_state_128])

# Class definition

For the first part of the simulation I need to build the blueprint for the system. This is the environment inside which the simulation takes place. It contains two classes:
 - Buffer class: which takes parts from one machine and put parts into another. It gives state messages when empty or full.
 - Machine class: the machines run in different states with differing speeds. This solves the problem of needing to fail and repair the machines

In [5]:
# Setup for buffer
@dataclass
class Buffer(object):
    """
    This is the class that defines the buffers of the packaging line. 
    The function of this class is to get parts from a machine before 
    it and to put parts into the machine after it.
    """
    def __init__(
        self,
        id: int,
        max_capacity: int = np.inf,
        current_parts: int = 0):
        
        self.id = id
        self.max_capacity = max_capacity
        self.current_parts = current_parts   
    
    def get_parts(self, amount: int):
        """
        This function make the buffer release parts to the machine after it. 
        It takes the minimum between the amount (which is called from the machine after it)
        and the current amount of parts in the buffer, and "gives" these to the machine after it.
        
        If the buffer is empty it says so.
        """
        result_out = min([amount, self.current_parts])
        self.current_parts -= result_out
        return result_out
    
    def put_parts(self, amount: int) -> int:
        """
        This function make the buffer receive parts to the machine after it. 
        It takes the minimum between the amount (which is called from the machine before it)
        and the current space in the buffer, and "takes" these from the machine before it.
        
        If the buffer is full it says so.
        """
        buffer_space = self.max_capacity - self.current_parts
        result_in = min([amount, buffer_space])
        self.current_parts += result_in
        return result_in

In [1]:
# Class setup for a Machine
@dataclass
class Machine(object):
    """
    A machine produces units at a fixed processing speed,
    takes units from a store before and puts units into a store after.
    """
    def __init__(
        self,
        env: simpy.Environment,
        name: str,
        machine_config: MachineConfig,
        initial_state: int,
        in_q: Buffer,
        out_q: Buffer,
    ):
        self.env = env
        self.name = name
        self.machine_config = machine_config
        self.state: State = machine_config.state(initial_state)
        self.in_q = in_q
        self.out_q = out_q
        self.count = 0
        self.produced = None
        self.idle_parts = 0
        self.tailback = 0
        self.lack = 0
        
    # Start the producing process
        self.process = env.process(self.produce())
   
    def produce(self):
        """
        Produce parts as long as the simulation runs.
        It is prone to breaking down and being repaired.
        """
        last_state_switch = self.env.now
        time_in_state = self.state.new_duration()
        # print(f"init {self.name} in state {self.state.id} for {time_in_state} minutes")
        while True:
            pspeed = self.get_speed()
            # Update count and what goes out and in buffers
            start_producing = self.in_q.get_parts(pspeed)
            # Check whether there is a lack
            if start_producing < pspeed and pspeed != 0:
                self.lack = 1
            else:
                self.lack = 0
            success_produced = self.out_q.put_parts(start_producing)
            # Check whether there is a tailback and set pspeed to 0
            if success_produced < pspeed and self.lack == 0:
                pspeed = 0
                self.tailback = 1
                self.idle_parts += start_producing - success_produced
            else:
                self.tailback = 0
            self.produced = success_produced
            self.count += self.produced
            # While machine has not been in state for given time
            if self.env.now == last_state_switch + time_in_state:
                self.state, time_in_state = self.get_next_state()
                last_state_switch = self.env.now
            event_list.append({"Time": self.env.now, "Machine": self.name, 
                               "State": self.state.id, "Set speed": pspeed, "Actual speed": self.produced, 
                               "Count": self.count, "Tailback": self.tailback, "Lack": self.lack, 'Simulation': None})                
            yield env.timeout(1)

    def get_speed(self):
        speed = 0
        if not (self.tailback or self.lack):
            speed = self.state.new_speed()
        return speed
        

    def get_next_state(self) -> Tuple[int, int]:
        # Select a state at random with given probabilities
        next_state_id = self.state.next_state_id()
        next_state = self.machine_config.state(next_state_id)
        # Find time in state (Complete and fill in correct params)
        time_in_state = next_state.new_duration()
        return next_state, time_in_state

NameError: name 'dataclass' is not defined

## Environment and Process setup

In [10]:
# Time for the simulation to run
run_time = 481

def run_sim(i):     
    # The buffers
    buffer_0 = Buffer("Buffer_0", current_parts = np.inf)
    buffer_1 = Buffer("Buffer_1", max_capacity = 3256) # 
    buffer_2 = Buffer("Buffer_2", max_capacity = 3400) # 
    buffer_3 = Buffer("Buffer_3", max_capacity = 3366) # 
    buffer_4 = Buffer("Buffer_4", max_capacity = 3420) # 
    buffer_5 = Buffer("Buffer_5", max_capacity = 3530) # 
    buffer_6 = Buffer("Buffer_6", max_capacity = 35612) # 
    buffer_7 = Buffer("Buffer_7")
    
    # The machines
    depalletizer = Machine(env, 'Depalletizer', Depalletizer, 128, buffer_0, buffer_1)
    filler = Machine(env, 'Filler', Filler, 128, buffer_1, buffer_2)
    pasteurizer = Machine(env, 'Pasteurizer', Pasteurizer, 128, buffer_2, buffer_3)
    labeler_1 = Machine(env, 'Labeler_1', Labeler1, 128, buffer_3, buffer_4)
    labeler_2 = Machine(env, 'Labeler_2', Labeler2, 128, buffer_3, buffer_4)
    capper = Machine(env, 'Capper', Capper, 128, buffer_4, buffer_5)
    packer = Machine(env, 'Packer', Packer, 128, buffer_5, buffer_6)
    palletizer_1 = Machine(env, 'Palletizer_1', Palletizer1, 128, buffer_6, buffer_7)
    palletizer_2 = Machine(env, 'Palletizer_2', Palletizer2, 128, buffer_6, buffer_7)
    
    # Execute
    env.run(until = run_time)
    return event_list

In [11]:
event_list = []
begin_time = datetime.now()
num_sim = 10000
for i in range(num_sim):
    print(f'---------- STARTING SIMULATION {i+1} ----------')
    sim_start = datetime.now() # To see how long each simulation takes
    event_list.append({"Time": 0.00, "Machine": "All", "State": "Simulation start", 
                       "Set speed": 0, "Actual speed": 0, "Count": 0, "Tailback": 0, 
                        "Lack": 0, 'Simulation': int(i+1)})
    # Create environment and start the setup process
    env = simpy.Environment()
    run_sim(i)
    print(datetime.now() - sim_start)

print(f'The total runtime for {num_sim} is {datetime.now() - begin_time}') # To see the total time of all simulations

---------- STARTING SIMULATION 1 ----------
0:00:00.042994
---------- STARTING SIMULATION 2 ----------
0:00:00.040187
---------- STARTING SIMULATION 3 ----------
0:00:00.036452
---------- STARTING SIMULATION 4 ----------
0:00:00.033373
---------- STARTING SIMULATION 5 ----------
0:00:00.033939
---------- STARTING SIMULATION 6 ----------
0:00:00.035606
---------- STARTING SIMULATION 7 ----------
0:00:00.036234
---------- STARTING SIMULATION 8 ----------
0:00:00.036749
---------- STARTING SIMULATION 9 ----------
0:00:00.034243
---------- STARTING SIMULATION 10 ----------
0:00:00.037437
---------- STARTING SIMULATION 11 ----------
0:00:00.035341
---------- STARTING SIMULATION 12 ----------
0:00:00.034260
---------- STARTING SIMULATION 13 ----------
0:00:00.038275
---------- STARTING SIMULATION 14 ----------
0:00:00.042742
---------- STARTING SIMULATION 15 ----------
0:00:00.041441
---------- STARTING SIMULATION 16 ----------
0:00:00.036034
---------- STARTING SIMULATION 17 ----------
0:00

0:00:00.035960
---------- STARTING SIMULATION 147 ----------
0:00:00.036434
---------- STARTING SIMULATION 148 ----------
0:00:00.049382
---------- STARTING SIMULATION 149 ----------
0:00:00.042959
---------- STARTING SIMULATION 150 ----------
0:00:00.034525
---------- STARTING SIMULATION 151 ----------
0:00:00.038815
---------- STARTING SIMULATION 152 ----------
0:00:00.034871
---------- STARTING SIMULATION 153 ----------
0:00:00.040504
---------- STARTING SIMULATION 154 ----------
0:00:00.036978
---------- STARTING SIMULATION 155 ----------
0:00:00.037727
---------- STARTING SIMULATION 156 ----------
0:00:00.043158
---------- STARTING SIMULATION 157 ----------
0:00:00.036026
---------- STARTING SIMULATION 158 ----------
0:00:00.039733
---------- STARTING SIMULATION 159 ----------
0:00:00.035606
---------- STARTING SIMULATION 160 ----------
0:00:00.039353
---------- STARTING SIMULATION 161 ----------
0:00:00.037897
---------- STARTING SIMULATION 162 ----------
0:00:00.038409
---------

0:00:00.034171
---------- STARTING SIMULATION 289 ----------
0:00:00.033527
---------- STARTING SIMULATION 290 ----------
0:00:00.035772
---------- STARTING SIMULATION 291 ----------
0:00:00.042001
---------- STARTING SIMULATION 292 ----------
0:00:00.041365
---------- STARTING SIMULATION 293 ----------
0:00:00.038543
---------- STARTING SIMULATION 294 ----------
0:00:00.045854
---------- STARTING SIMULATION 295 ----------
0:00:00.038305
---------- STARTING SIMULATION 296 ----------
0:00:00.035142
---------- STARTING SIMULATION 297 ----------
0:00:00.033361
---------- STARTING SIMULATION 298 ----------
0:00:00.034950
---------- STARTING SIMULATION 299 ----------
0:00:00.037740
---------- STARTING SIMULATION 300 ----------
0:00:00.034843
---------- STARTING SIMULATION 301 ----------
0:00:00.035337
---------- STARTING SIMULATION 302 ----------
0:00:00.033859
---------- STARTING SIMULATION 303 ----------
0:00:00.032867
---------- STARTING SIMULATION 304 ----------
0:00:00.033263
---------

0:00:00.033717
---------- STARTING SIMULATION 428 ----------
0:00:00.032030
---------- STARTING SIMULATION 429 ----------
0:00:00.032596
---------- STARTING SIMULATION 430 ----------
0:00:00.035730
---------- STARTING SIMULATION 431 ----------
0:00:00.033932
---------- STARTING SIMULATION 432 ----------
0:00:00.036328
---------- STARTING SIMULATION 433 ----------
0:00:00.031799
---------- STARTING SIMULATION 434 ----------
0:00:00.033184
---------- STARTING SIMULATION 435 ----------
0:00:00.032572
---------- STARTING SIMULATION 436 ----------
0:00:00.034283
---------- STARTING SIMULATION 437 ----------
0:00:00.033791
---------- STARTING SIMULATION 438 ----------
0:00:00.032477
---------- STARTING SIMULATION 439 ----------
0:00:00.031705
---------- STARTING SIMULATION 440 ----------
0:00:00.036570
---------- STARTING SIMULATION 441 ----------
0:00:00.033685
---------- STARTING SIMULATION 442 ----------
0:00:00.032152
---------- STARTING SIMULATION 443 ----------
0:00:00.031312
---------

0:00:00.041571
---------- STARTING SIMULATION 567 ----------
0:00:00.030694
---------- STARTING SIMULATION 568 ----------
0:00:00.042540
---------- STARTING SIMULATION 569 ----------
0:00:00.036192
---------- STARTING SIMULATION 570 ----------
0:00:00.037421
---------- STARTING SIMULATION 571 ----------
0:00:00.039017
---------- STARTING SIMULATION 572 ----------
0:00:00.052389
---------- STARTING SIMULATION 573 ----------
0:00:00.034613
---------- STARTING SIMULATION 574 ----------
0:00:00.039797
---------- STARTING SIMULATION 575 ----------
0:00:00.043608
---------- STARTING SIMULATION 576 ----------
0:00:00.048638
---------- STARTING SIMULATION 577 ----------
0:00:00.038285
---------- STARTING SIMULATION 578 ----------
0:00:00.035372
---------- STARTING SIMULATION 579 ----------
0:00:00.037799
---------- STARTING SIMULATION 580 ----------
0:00:00.038287
---------- STARTING SIMULATION 581 ----------
0:00:00.037933
---------- STARTING SIMULATION 582 ----------
0:00:00.036119
---------

...

0:00:00.040005
---------- STARTING SIMULATION 9343 ----------
0:00:00.038628
---------- STARTING SIMULATION 9344 ----------
0:00:00.036532
---------- STARTING SIMULATION 9345 ----------
0:00:00.049106
---------- STARTING SIMULATION 9346 ----------
0:00:00.041191
---------- STARTING SIMULATION 9347 ----------
0:00:00.040615
---------- STARTING SIMULATION 9348 ----------
0:00:00.041717
---------- STARTING SIMULATION 9349 ----------
0:00:00.066007
---------- STARTING SIMULATION 9350 ----------
0:00:00.039819
---------- STARTING SIMULATION 9351 ----------
0:00:00.042824
---------- STARTING SIMULATION 9352 ----------
0:00:00.044345
---------- STARTING SIMULATION 9353 ----------
0:00:00.038597
---------- STARTING SIMULATION 9354 ----------
0:00:00.044734
---------- STARTING SIMULATION 9355 ----------
0:00:00.093239
---------- STARTING SIMULATION 9356 ----------
0:00:00.039404
---------- STARTING SIMULATION 9357 ----------
0:00:00.037834
---------- STARTING SIMULATION 9358 ----------
0:00:00.

0:00:00.035420
---------- STARTING SIMULATION 9487 ----------
0:00:00.036532
---------- STARTING SIMULATION 9488 ----------
0:00:00.034947
---------- STARTING SIMULATION 9489 ----------
0:00:00.031432
---------- STARTING SIMULATION 9490 ----------
0:00:00.036969
---------- STARTING SIMULATION 9491 ----------
0:00:00.036648
---------- STARTING SIMULATION 9492 ----------
0:00:00.044901
---------- STARTING SIMULATION 9493 ----------
0:00:00.035504
---------- STARTING SIMULATION 9494 ----------
0:00:00.035092
---------- STARTING SIMULATION 9495 ----------
0:00:00.035283
---------- STARTING SIMULATION 9496 ----------
0:00:00.034787
---------- STARTING SIMULATION 9497 ----------
0:00:00.034880
---------- STARTING SIMULATION 9498 ----------
0:00:00.035482
---------- STARTING SIMULATION 9499 ----------
0:00:00.036403
---------- STARTING SIMULATION 9500 ----------
0:00:00.037040
---------- STARTING SIMULATION 9501 ----------
0:00:00.037375
---------- STARTING SIMULATION 9502 ----------
0:00:00.

0:00:00.040745
---------- STARTING SIMULATION 9620 ----------
0:00:00.039009
---------- STARTING SIMULATION 9621 ----------
0:00:00.042355
---------- STARTING SIMULATION 9622 ----------
0:00:00.037634
---------- STARTING SIMULATION 9623 ----------
0:00:00.042902
---------- STARTING SIMULATION 9624 ----------
0:00:00.039202
---------- STARTING SIMULATION 9625 ----------
0:00:00.039378
---------- STARTING SIMULATION 9626 ----------
0:00:00.036269
---------- STARTING SIMULATION 9627 ----------
0:00:00.037153
---------- STARTING SIMULATION 9628 ----------
0:00:00.038198
---------- STARTING SIMULATION 9629 ----------
0:00:00.035043
---------- STARTING SIMULATION 9630 ----------
0:00:00.035280
---------- STARTING SIMULATION 9631 ----------
0:00:00.039507
---------- STARTING SIMULATION 9632 ----------
0:00:00.041478
---------- STARTING SIMULATION 9633 ----------
0:00:00.039493
---------- STARTING SIMULATION 9634 ----------
0:00:00.041589
---------- STARTING SIMULATION 9635 ----------
0:00:00.

0:00:00.039110
---------- STARTING SIMULATION 9756 ----------
0:00:00.038529
---------- STARTING SIMULATION 9757 ----------
0:00:00.043759
---------- STARTING SIMULATION 9758 ----------
0:00:00.039378
---------- STARTING SIMULATION 9759 ----------
0:00:00.042445
---------- STARTING SIMULATION 9760 ----------
0:00:00.042657
---------- STARTING SIMULATION 9761 ----------
0:00:00.035868
---------- STARTING SIMULATION 9762 ----------
0:00:00.038920
---------- STARTING SIMULATION 9763 ----------
0:00:00.038279
---------- STARTING SIMULATION 9764 ----------
0:00:00.998017
---------- STARTING SIMULATION 9765 ----------
0:00:00.035912
---------- STARTING SIMULATION 9766 ----------
0:00:00.031529
---------- STARTING SIMULATION 9767 ----------
0:00:00.031238
---------- STARTING SIMULATION 9768 ----------
0:00:00.033247
---------- STARTING SIMULATION 9769 ----------
0:00:00.034787
---------- STARTING SIMULATION 9770 ----------
0:00:00.035315
---------- STARTING SIMULATION 9771 ----------
0:00:00.

0:00:00.035923
---------- STARTING SIMULATION 9896 ----------
0:00:00.034480
---------- STARTING SIMULATION 9897 ----------
0:00:00.040285
---------- STARTING SIMULATION 9898 ----------
0:00:00.042360
---------- STARTING SIMULATION 9899 ----------
0:00:00.038748
---------- STARTING SIMULATION 9900 ----------
0:00:00.039416
---------- STARTING SIMULATION 9901 ----------
0:00:00.038922
---------- STARTING SIMULATION 9902 ----------
0:00:00.038731
---------- STARTING SIMULATION 9903 ----------
0:00:00.039434
---------- STARTING SIMULATION 9904 ----------
0:00:00.043860
---------- STARTING SIMULATION 9905 ----------
0:00:00.036302
---------- STARTING SIMULATION 9906 ----------
0:00:00.037345
---------- STARTING SIMULATION 9907 ----------
0:00:00.038533
---------- STARTING SIMULATION 9908 ----------
0:00:00.038189
---------- STARTING SIMULATION 9909 ----------
0:00:00.038414
---------- STARTING SIMULATION 9910 ----------
0:00:00.037769
---------- STARTING SIMULATION 9911 ----------
0:00:00.

In [12]:
begin_time = datetime.now()
log = pd.DataFrame(event_list)
log['Simulation'].ffill(inplace = True)
log.to_csv('data/Sim data/sim_log_decrease_s1000.csv')
print(f'The total runtime for {num_sim} is {datetime.now() - begin_time}')

The total runtime for 10000 is 0:05:28.388947
